# LABELS

In [ ]:
import os
import re

In [ ]:
dataSetMLF_FilesPath=r'E:\UAspeesh dataset\mlf'
dataSetMLF_FilesName=[file for file in os.listdir(dataSetMLF_FilesPath)if file.endswith('.mlf')]

In [ ]:
dataSetTxtFilesPath=r'C:\Users\Ahmed Salah\Desktop\files'
dataSetTxtFilesName=[file for file in os.listdir(dataSetTxtFilesPath)if file.endswith('.txt')]

In [ ]:
dataSetMLF_FilesName

In [ ]:
dataSetTxtFilesName

## Convert MLF Files To one Dict {FileName:Word}

In [ ]:
def convertFromMlfToText(mlf_files,dataSetMLF_FilesPath):
    allmlf_FilesInDict,errorReadMlfFiles={},[]
    for mlf_File in mlf_files:
        try: 
            with open(dataSetMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
                mlf_FileBufferWords=mlf_FileBuffer.readlines()
            CounterOfmlf_FileBufferWords=0
            for line in mlf_FileBufferWords:
                CounterOfmlf_FileBufferWords+=1
                if re.match(r'"\*/\w+.lab"',line):
                    allmlf_FilesInDict[re.sub('.lab','.wav',re.sub('"\n','',re.sub(r'"\*/','',line)))]=mlf_FileBufferWords[CounterOfmlf_FileBufferWords]
        except:
            errorReadMlfFiles.append(mlf_File)

    return allmlf_FilesInDict,errorReadMlfFiles

In [ ]:
%%time
allmlf_FilesInDict,errorReadMlfFiles=convertFromMlfToText(dataSetMLF_FilesName,dataSetMLF_FilesPath)

In [ ]:
print(len(allmlf_FilesInDict))
print(len(errorReadMlfFiles))

## convert all mlf files to one txt file as Dict (file:word)

In [ ]:
def convertDictToFile(Dict,nameOfFile): 
    if not os.path.isdir('files'):
        os.mkdir('files')
    with open('files/'+nameOfFile+'.txt','w') as newLabelFile:
            for key in Dict:
                newLabelFile.writelines(key+':'+Dict[key].strip()+'\n')

In [ ]:
convertDictToFile(allmlf_FilesInDict,'labels')

## Read From Text file {FileName:Word}

In [ ]:
def readLabelsFromTextFileAsDict(labelsFiles,DataSetPath):
    allLabelsFilesAsDict,errorReadLabelFiles={},[]
    for File in labelsFiles:
        try:
            with open(DataSetPath+'\\'+File,mode='r') as labelsFile:
                for line in sorted(labelsFile.read().split('\n')):
                    line=line.split(':')
                    allLabelsFilesAsDict[line[0]]=line[-1]
        except:
            errorReadLabelFiles.append(File)
    return allLabelsFilesAsDict,errorReadLabelFiles
    

In [ ]:
%%time
allLabelsFilesAsDict,errorReadLabelFiles=readLabelsFromTextFileAsDict(dataSetTxtFilesName,dataSetTxtFilesPath)

In [ ]:
print(len(allLabelsFilesAsDict))
print(len(errorReadLabelFiles))

## Convert MLF Files To Multi TXT Files (107100) in Folder (files)

In [ ]:
def convertFromMlfToTxtFiles(dataSetMLF_Files,dataSetMLF_FilesPath):
    if not os.path.isdir('files'):
        os.mkdir('files')
    ErrorFileNameInLineDict={}
    for mlf_File in dataSetMLF_Files: 
        with open(dataSetMLF_FilesPath+'\\'+mlf_File,mode='r',encoding='utf-8') as mlf_FileBuffer:
            mlf_FileBufferWords=mlf_FileBuffer.readlines()
        CounterOfmlf_FileBufferWords=0
        for line in mlf_FileBufferWords:
            CounterOfmlf_FileBufferWords+=1
            if re.match(r'"\*/\w+.lab"',line):
                try:
                    with open('files/'+re.sub('.lab"\n','',re.sub(r'"\*/','',line))+'.txt','w') as WordFile:
                        WordFile.write(mlf_FileBufferWords[CounterOfmlf_FileBufferWords])
                except:
                    ErrorFileNameInLineDict[mlf_File+line]=CounterOfmlf_FileBufferWords
    return ErrorFileNameInLineDict


## Print If There Erorr OR Not 

In [ ]:
print(ErrorFileNameInLineDict)

# AUDIO

In [ ]:
import os
dataSetMainFilesOfDataPath=r'G:\UASpeech\audio' #edit path
dataSetMainFilesOfDataName=[folder for folder in os.listdir(dataSetMainFilesOfDataPath)if folder.endswith('_audio')] #to get only Folders

In [ ]:
print(dataSetMainFilesOfDataName) # all Main Files in dataset

## get All Audio Files Paths

In [ ]:
def getAudioFilesPath(DatasetPath,dataSetMainFiles):
    dataSetAudioFilesPaths=[]
    for mainFile in dataSetMainFiles:#main Files as CF02
        for mainFileChiled in sorted(os.listdir(DatasetPath+'\\'+mainFile)):#git records in every main file 
            if mainFileChiled.endswith('.wav'):
                dataSetAudioFilesPaths.append(DatasetPath+'\\'+mainFile+'\\'+mainFileChiled)#append Path of file .wav
    return dataSetAudioFilesPaths


In [ ]:
%%time
dataSetAudioFilesPaths=getAudioFilesPath(dataSetMainFilesOfDataPath,dataSetMainFilesOfDataName)

In [ ]:
print(len(dataSetAudioFilesPaths))
print(dataSetAudioFilesPaths[0])
print(dataSetAudioFilesPaths[-1])


## Delete All Audio Files That Haven't Label

In [ ]:
def deleteAudioFilesWithoutLabel(dataSetAudioFilesPaths,allLabelsFilesAsDict):
    audioFilesHadDeleted=[]
    dataSetAudioFilesPathsCopy=dataSetAudioFilesPaths.copy()
    ErrorRemovedFile=[]
    for audioFilePath in dataSetAudioFilesPaths:
        if audioFileName not in allLabelsFilesAsDict:
            try:
                audioFilesHadDeleted.append(audioFilePath)
                os.remove(audioFilePath)
                dataSetAudioFilesPathsCopy.remove(audioFilePath)
            except:
                ErrorRemovedFile.append(audioFilePath)
    return dataSetAudioFilesPathsCopy,audioFilesHadDeleted,ErrorRemovedFile

In [ ]:
%%time
dataSetAudioFilesPaths,audioFilesHadDeleted,ErrorRemovedFile=deleteAudioFilesWithoutLabel(dataSetAudioFilesPaths,allLabelsFilesAsDict)

In [ ]:
print(len(dataSetAudioFilesPaths)) #all paths after update
print(len(audioFilesHadDeleted)) #if there file had remove
print(len(ErrorRemovedFile)) #if there file have error and didn't remove

## Delete All labels That Haven't Audio File

In [ ]:
def deleteLabelWithoutAudio(allLabelsFilesAsDict,dataSetAudioFilesPaths):
    labelLinesHadDeleted=[]
    dataSetAudioFilesNames=[audioFileName.split('\\')[-1] for audioFileName in dataSetAudioFilesPaths ]
    allLabelsFilesAsDictCopy=allLabelsFilesAsDict.copy()
    for mlf_FileWord in allLabelsFilesAsDict:
        if mlf_FileWord not in dataSetAudioFilesNames:
            labelLinesHadDeleted.append(mlf_FileWord)
            allLabelsFilesAsDictCopy.pop(mlf_FileWord)
    del(dataSetAudioFilesNames)
    return allLabelsFilesAsDictCopy,labelLinesHadDeleted

In [ ]:
%%time
allLabelsFilesAsDict,labelLinesHadDeleted=deleteLabelWithoutAudio(allLabelsFilesAsDict,dataSetAudioFilesPaths)

In [ ]:
print(len(allLabelsFilesAsDict)) # labels after update
print(len(labelLinesHadDeleted)) #lines that deleted

In [ ]:
convertDictToFile(allLabelsFilesAsDict,'labels') #convert labels Dict to file after update

# Rename 

In [ ]:
import re
def renameAllFiles(DatasetPath,dataSetMainFiles,LabelsFilesAsDict):
    newLabelsDict,formOldeToNew,errorRenameFile={},{},[]
    for mainFile in dataSetMainFiles:#main Files as CF02
        audioNumber=0
        for mainFileChiled in os.listdir(DatasetPath+'\\'+mainFile):#git records in every main file 
            if mainFileChiled.endswith('.wav'):
                audioNumber+=1
                newText=re.sub('_audio','_',mainFile)
                for zeros in range((4-len(str(audioNumber)))):
                    newText+='0'
                newFileName=newText+ str(audioNumber) + '.wav'
                oldFilePath=( DatasetPath+'\\'+mainFile+'\\'+mainFileChiled)
                newFilePath=( DatasetPath+'\\'+mainFile+'\\'+newFileName)
                newLabelsDict[newFileName]=LabelsFilesAsDict[mainFileChiled]
                formOldeToNew[mainFileChiled]=newFileName
                try:
                    os.rename(oldFilePath,newFilePath)
                    print(mainFileChiled)
                except:
                    errorRenameFile.append(mainFileChiled)
    return newLabelsDict,formOldeToNew,errorRenameFile
                

In [ ]:
%%time
allLabelsFilesAsDict,formOldeToNew,errorRenameFile=renameAllFiles(dataSetMainFilesOfDataPath,dataSetMainFilesOfDataName,allLabelsFilesAsDict)

In [ ]:
print(len(errorRenameFile))


In [ ]:
convertDictToFile(allLabelsFilesAsDict,'labels')
convertDictToFile(formOldeToNew,'formOldeToNew')
